In [ ]:
import os

path = "/Users/md.minhajulislampranto/Desktop/pgn"
files = [f for f in os.listdir("pgn") if f.endswith(".pgn")]


In [31]:
from chess import pgn

def load_pgn(file_path):
    games=[]
    with open(file_path,'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

In [33]:
from tqdm import tqdm 

In [35]:
import os
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files), 24)
games = []

for i, file in enumerate(tqdm(files)):
    games.extend(load_pgn(os.path.join("pgn", file)))
    if i + 1 >= LIMIT_OF_FILES:
        break


 95%|█████████▌| 21/22 [00:08<00:00,  2.52it/s]


In [ ]:
len(games)

In [39]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore
import time

In [45]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix


def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y


def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [49]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=50, validation_split=0.1, batch_size=64)
model.save("models/TF_50EPOCHS.keras")

In [52]:
model.save("my_model_epoc_50_7014.keras")

In [57]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))


def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

In [59]:
# Create a chess board (start position)
board = Board()

In [ ]:
# Display the board before prediction
print("Board before prediction:")
print(board)

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
print(board)

In [272]:
print(str(pgn.Game.from_board(board)))

[Event "?"]
[Site "?"]
[Date "????.??.??"]
[Round "?"]
[White "?"]
[Black "?"]
[Result "1/2-1/2"]

1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. Nc3 Nc6 6. Bg5 e6 7. Qd2 a6 8. O-O-O Be7 9. Nxc6 bxc6 10. Qd4 c5 11. Qd2 O-O 12. f4 h6 13. Bh4 d5 14. exd5 exd5 15. Bd3 Be6 16. f5 Bd7 17. Rhe1 Rc8 18. Re5 d4 19. Nd5 Nxd5 20. Bxe7 Qxe7 21. c3 Nxc3 22. bxc3 Qxe5 23. cxd4 cxd4+ 24. Kb2 Rc1 25. Qxc1 Re8 26. Qc8 Qd6 27. Qd8 Rxd8 28. Rd2 f6 29. Be4 Kf7 30. Ka1 Qd5 31. Bxd5+ Ke7 32. Bb7 d3 33. Be4 Rg8 34. Rb2 d2 35. Rb7 Kd8 36. Rxd7+ Kxd7 37. h4 a5 38. h5 Ke7 39. Bd3 Kf8 40. Bc4 Ke7 41. Bb3 Kf8 42. Ba4 d1=Q+ 43. Kb2 Qc1+ 44. Kb3 Qc3+ 45. Kxc3 Ke7 46. Be8 Rf8 47. Kc4 Kxe8 48. Kc5 Kd7 49. Kd5 a4 50. Kd4 a3 51. Ke4 Kd6 52. Kd4 Kc6 53. Kc4 Kd7 54. Kd5 Kc7 55. Ke4 Kd6 56. Kd4 Kc6 57. Kc4 Kd7 58. Kd5 Kc7 59. Ke4 Kd6 60. Kd4 Kc6 61. Kc4 Kd7 62. Kd5 Kc7 63. Ke4 Kd6 64. Kd4 Kc6 65. Kc4 Kd7 66. Kd5 Kc7 67. Ke4 Kd6 68. Kd4 Kc6 69. Kc4 Kd7 70. Kd5 Kc7 71. Ke4 Kd6 72. Kd4 Kc6 73. Kc4 Kd7 74. Kd5 Kc7 75. Ke4 Kd6 